# **Titanic**

## **Set up**

In [1]:
import sklearn
assert sklearn.__version__ >= "0.20"

import os
import numpy as np
import pandas as pd
import warnings

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = '.'
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)



warnings.filterwarnings('ignore')

## **Load Titanic Data**

In [4]:
import os
import zipfile
import urllib

downlaod_root = "https://raw.githubusercontent.com/dfongang/handson-ml2/master/"
data_path = os.path.join("datasets")
data_url = downlaod_root + "chapter_03/datasets/titanic.zip"

def fetch_titanic_data(data_url=data_url, data_path=data_path):
    os.makedirs(data_path, exist_ok=True)
    zip_path = os.path.join(data_path, 'titanic.zip')
    urllib.request.urlretrieve(data_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zp:
        zp.extractall(path=data_path)

In [5]:
fetch_titanic_data()

### Load the data